In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [3]:
import pymysql
import json
import warnings
import DB
warnings.filterwarnings(action='ignore')

In [179]:
db = DB.MYDB()
db.cursor.execute("select * from team_record where home_away = 'H' and gmkey >= 'S17' order by gmkey")
team_hdata = pd.DataFrame(db.cursor.fetchall())
team_hdata = team_hdata.drop(team_hdata[team_hdata['gmkey'].isin(['S37G13N10', 'S37G13N2', 'S39G13N2', 'S39G13N5'])].index)
team_hdata = team_hdata.reset_index(drop=True)
db.cursor.execute("select * from team_record where home_away = 'A' and gmkey >= 'S17' order by gmkey")
team_adata = pd.DataFrame(db.cursor.fetchall())
team_adata = team_adata.drop(team_adata[team_adata['gmkey'].isin(['S37G13N10', 'S37G13N2', 'S39G13N2', 'S39G13N5'])].index)
team_adata = team_adata.reset_index(drop=True)
db.db.close()


Connect!


In [3]:
def calculate_data(h_data, a_data):
  output_data = pd.DataFrame()
  output_data['gmkey'] = h_data['gmkey']
  output_data['ast'] = h_data['ast'] / h_data['fgt']
  # 2점
  output_data['twop'] = h_data['fg'] / h_data['fgA']
  # 3점
  output_data['threep'] = h_data['threep'] / h_data['threepA']
  # 자유투
  output_data['freep'] = h_data['ft'] / h_data['ftA']
  # 공격 리바
  output_data['offr'] = h_data['offr'] / (h_data['offr'] + a_data['defr'])
  # 속공
  # output_data['tfb'] = h_data['tfb'] / h_data['fgt']
  # 벤치득점
  # output_data['bench'] = h_data['benchScoreCn'] / h_data['score']
  # 속공득점
  # output_data['tfbp'] = h_data['fbScoreCn'] / h_data['score']
  # 페인트존 득점
  output_data['ppp'] = h_data['pp'] / h_data['ppA']
  # 덩크
  output_data['dk'] = h_data['dk'] / h_data['dkA']
  # 최다 연속 득점 비율
  # output_data['maxContiScore'] = h_data['maxContiScoreCn'] / h_data['score']
  # 최다 리드 점수차
  # output_data['maxLeadScore'] = (h_data['score'] - a_data['score']) / h_data['maxLeadScoreCn']
  # 득점 우위 시간
  # output_data['scoreHighTime'] = h_data['scoreHighTimeCn'] / 2400
  # 세컨찬스 득점
  # output_data['secChanceScore'] = h_data['secChanceScoreCn'] / h_data['score']
  # 턴오버
  # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])
  # output_data['to'] = (h_data['to'] + h_data['tto']) / (h_data['to'] + h_data['tto'] + a_data['to'] + a_data['tto'])

  output_data['defr'] = h_data['defr'] / (a_data['offr'] + h_data['defr'])
  output_data['stl'] = h_data['stl'] / (h_data['stl'] + a_data['stl'])
  output_data['bs'] = h_data['bs'] / a_data['fgtA']
  output_data['foul'] = h_data['foul'] / (h_data['foul'] + a_data['foul'])
  output_data['tf'] = h_data['tf'] / (h_data['tf'] + a_data['tf'])
  output_data['gd'] = h_data['gd'] / (h_data['gd'] + a_data['gd'])

  output_data['score'] = h_data['score']
  
  # output_data['teamR'] = h_data['teamR'] / (h_data['teamR'] + a_data['teamR'])
  # result.append(output_data, ignore_index=True)
  # print(output_data)
  # result = pd.concat([result, pd.DataFrame([output_data])], ignore_index=True)
  return output_data

In [4]:
def team_player_concat(team, player, team_rate):
  player_rate = 1 - team_rate
  output_data = pd.DataFrame()
  output_data['gmkey'] = team['gmkey']
  output_data['ast'] = team_rate * team['ast'] + player_rate * player['ast']
  # 2점
  output_data['twop'] = team_rate * team['twop'] + player_rate * player['twop']
  # 3점
  output_data['threep'] = team_rate * team['threep'] + player_rate * player['threep']
  # 자유투
  output_data['freep'] = team_rate * team['freep'] + player_rate * player['freep']
  # 공격 리바
  output_data['offr'] = team_rate * team['offr'] + player_rate * player['offr']
  # 속공
  # output_data['tfb'] = team['tfb'] / team['fgt']
  # 벤치득점
  # output_data['bench'] = team['benchScoreCn'] / team['score']
  # 속공득점
  # output_data['tfbp'] = team['fbScoreCn'] / team['score']
  # 페인트존 득점
  output_data['ppp'] = team_rate * team['ppp'] + player_rate * player['ppp']
  # 덩크
  output_data['dk'] = team_rate * team['dk'] + player_rate * player['dk']
  # 최다 연속 득점 비율
  # output_data['maxContiScore'] = team['maxContiScoreCn'] / team['score']
  # 최다 리드 점수차
  # output_data['maxLeadScore'] = (team['score'] - player['score']) / team['maxLeadScoreCn']
  # 득점 우위 시간
  # output_data['scoreHighTime'] = team['scoreHighTimeCn'] / 2400
  # 세컨찬스 득점
  # output_data['secChanceScore'] = team['secChanceScoreCn'] / team['score']
  # 턴오버
  # output_data['to'] = (team['to'] + team['tto']) / (team['to'] + team['tto'] + player['to'] + player['tto'])
  # output_data['to'] = (team['to'] + team['tto']) / (team['to'] + team['tto'] + player['to'] + player['tto'])

  output_data['defr'] = team_rate * team['defr'] + player_rate * player['defr']
  output_data['stl'] = team_rate * team['stl'] + player_rate * player['stl']
  output_data['bs'] = team_rate * team['bs'] + player_rate * player['bs']
  output_data['foul'] = team_rate * team['foul'] + player_rate * player['foul']
  output_data['tf'] = team_rate * team['tf'] + player_rate * player['tf']
  output_data['gd'] = team_rate * team['gd'] + player_rate * player['gd']

  output_data['score'] = team['score']
  
  # output_data['teamR'] = team['teamR'] / (team['teamR'] + player['teamR'])
  # result.append(output_data, ignore_index=True)
  # print(output_data)
  # result = pd.concat([result, pd.DataFrame([output_data])], ignore_index=True)
  return output_data

In [182]:
def make_model(data):
    h_data = pd.DataFrame()
    a_data = pd.DataFrame()
    # 어시스트 -> 어시스트 / 필드골 성공
    h_data['ast'] = data['h_ast'] / data['h_fgt']
    a_data['ast'] = data['a_ast'] / data['a_fgt']
    # 2점
    h_data['twop'] = data['h_fg'] / data['h_fgA']
    a_data['twop'] = data['a_fg'] / data['a_fgA']

    # 3점
    h_data['threep'] = data['h_threep'] / data['h_threepA']
    a_data['threep'] = data['a_threep'] / data['a_threepA']

    # 자유투
    h_data['freep'] = data['h_ft'] / data['h_ftA']
    a_data['freep'] = data['a_ft'] / data['a_ftA']

    # 공격 리바
    h_data['offr'] = data['h_offr'] / (data['h_offr'] + data['a_defr'])
    a_data['offr'] = data['a_offr'] / (data['a_offr'] + data['h_defr'])
    # 속공
    h_data['tfb'] = data['h_tfb'] / data['h_fgt']
    a_data['tfb'] = data['a_tfb'] / data['a_fgt']

    # 벤치득점
    h_data['bench'] = data['h_benchScoreCn'] / data['h_score']
    a_data['bench'] = data['a_benchScoreCn'] / data['a_score']


    # 속공득점
    h_data['tfbp'] = data['h_fbScoreCn'] / data['h_score']
    a_data['tfbp'] = data['a_fbScoreCn'] / data['a_score']

    # 페인트존 득점
    h_data['ppp'] = data['h_pp'] / data['h_ppA']
    a_data['ppp'] = data['a_pp'] / data['a_ppA']

    # 덩크
    h_data['dk'] = data['h_dk'] / data['h_dkA']
    a_data['dk'] = data['a_dk'] / data['a_dkA']

    # 최다 연속 득점 비율
    h_data['maxContiScore'] = data['h_maxContiScoreCn'] / data['h_score']
    a_data['maxContiScore'] = data['a_maxContiScoreCn'] / data['a_score']

    # 최다 리드 점수차
    # h_data['maxLeadScore'] = (data['h_score'] - data['a_score']) / data['h_maxLeadScoreCn']
    # a_data['maxLeadScore'] = (data['a_score'] - data['h_score']) / data['a_maxLeadScoreCn']
    # h_data[h_data['maxLeadScore'] < -1]

    # 득점 우위 시간
    h_data['scoreHighTime'] = data['h_scoreHighTimeCn'] / 2400
    a_data['scoreHighTime'] = data['a_scoreHighTimeCn'] / 2400

    # 세컨찬스 득점
    h_data['secChanceScore'] = data['h_secChanceScoreCn'] / data['h_score']
    a_data['secChanceScore'] = data['a_secChanceScoreCn'] / data['a_score']

    # 턴오버
    h_data['to'] = (data['h_to'] + data['h_tto']) / (data['h_to'] + data['h_tto'] + data['a_to'] + data['a_tto'])
    a_data['to'] = (data['a_to'] + data['a_tto']) / (data['h_to'] + data['h_tto'] + data['a_to'] + data['a_tto'])

    h_data['defr'] = data['h_defr'] / (data['a_offr'] + data['h_defr'])
    a_data['defr'] = data['a_defr'] / (data['h_offr'] + data['a_defr'])
    h_data['stl'] = data['h_stl'] / (data['h_stl'] + data['a_stl'])
    a_data['stl'] = data['a_stl'] / (data['h_stl'] + data['a_stl'])
    h_data['bs'] = data['h_bs'] / data['a_fgtA']
    a_data['bs'] = data['a_bs'] / data['h_fgtA']
    h_data['foul'] = data['h_foul'] / (data['h_foul'] + data['a_foul'])
    a_data['foul'] = data['a_foul'] / (data['h_foul'] + data['a_foul'])
    h_data['tf'] = data['h_tf'] / (data['h_tf'] + data['a_tf'])
    a_data['tf'] = data['a_tf'] / (data['h_tf'] + data['a_tf'])
    h_data['gd'] = data['h_gd'] / (data['h_gd'] + data['a_gd'])
    a_data['gd'] = data['a_gd'] / (data['h_gd'] + data['a_gd'])
    h_data['teamR'] = data['h_teamR'] / (data['h_teamR'] + data['a_teamR'])
    a_data['teamR'] = data['a_teamR'] / (data['h_teamR'] + data['a_teamR'])

    # h_data = h_data.fillna(0)
    # h_data = h_data.replace([np.inf, -np.inf], 0)
    h_data['score'] = data['h_score']
    # h_data['gmkey'] = data['gmkey']

    # a_data = a_data.fillna(0)
    # a_data = a_data.replace([np.inf, -np.inf], 0)
    a_data['score'] = data['a_score']
    # a_data['gmkey'] = data['gmkey']
    return pd.concat([h_data, a_data], ignore_index=True)

In [183]:
#seasonKey Example : S19G01
def get_keyplayer(seasonKey, team):
    result = set()
    season = str(int(seasonKey[1:3]) -2)
    league = seasonKey[4:6]
    seasonKey = 'S' + season + 'G' + league
    db.cursor.execute('''SELECT *
                        FROM player_avg_record
                        WHERE seasonCode LIKE 'S{}%' and
                        pcode IN (SELECT pcode
                                    FROM player
                                    WHERE tcode = '{}' and seasonCode = '{}');'''.format(season, team, season))
    data = pd.DataFrame(db.cursor.fetchall())
    if data.size != 0:
        score_top_player = data.sort_values('score',ascending=False)['pcode'][:3]
        ast_top_player = data.sort_values('ast')['pcode'][:3]
        rb_top_player = data.sort_values('rb')['pcode'][:3]
        for player in score_top_player:
            result.add(player)
        # result.add(score_top_player[0])
        for player in ast_top_player:
            if player in result:
                continue
            result.add(player)
            # break
        for player in rb_top_player:
            if player in result:
                continue
            result.add(player)
            # break
        return result     
    else:
        return None

In [184]:
def get_season_keyplayer(season):
    # db = DB.MYDB()
    result = dict()
    seasonCode = 'S' + str(season) + 'G01'
    for tcode in tcodes:
        tcode = str(tcode['tcode'])
        # if tcode == '75':
            # tcode = '29'
        # db.cursor.execute('''SELECT *
        #                 FROM player_avg_record
        #                 WHERE seasonCode LIKE 'S{}%' and
        #                 pcode IN (SELECT pcode
        #                             FROM player
        #                             WHERE tcode = '{}' and seasonCode = '{}');'''.format(season - 2, tcode, season))
        db.cursor.execute('''SELECT *
                        FROM (
                            SELECT p.*, rank() over(partition by p.pcode order by p.seasonCode desc) as a
                            FROM player_avg_record as p
                            WHERE seasonCode < 'S{}' and
                                pcode IN (SELECT pcode
                                            FROM player
                                            WHERE tcode = '{}' and seasonCode = '{}')
                            ) as rankrow
                        where rankrow.a <=1;
        '''.format(seasonCode, tcode, season))
        data = pd.DataFrame(db.cursor.fetchall())
        if data.size != 0:
            score_top_player = data.sort_values('score',ascending=False)['pcode'][:1]
            ast_top_player = data.sort_values('ast', ascending=False)['pcode'][:1]
            rb_top_player = data.sort_values('rb', ascending=False)['pcode'][:1]
            result[tcode] = set(score_top_player) | set(ast_top_player) | set(rb_top_player)
    return result

In [185]:
def get_recent_players(players, gameDate, db):
    # df = pd.DataFrame()
    df = []
    # db = DB.MYDB()
    # for player in players:
        # db.cursor.execute("""
        #                 SELECT b.*, a.gameDate 
        #                 FROM game_meta a, player_record b
        #                 WHERE a.gmkey = b.gmkey and b.pcode = '{}' and b.gmkey < '{}'
        #                 ORDER BY gameDate desc
        #                 LIMIT 5""".format(player, gameCode))
    if len(players) == 1:
        players.add('0')
    db.cursor.execute("""
                        select *
                        from (
                            SELECT b.*, a.gameDate, rank() over(partition by b.pcode order by gameDate desc) as a
                            FROM game_meta a, player_record b
                            WHERE a.gmkey = b.gmkey and a.gameDate < '{}'
                                and b.pcode in {}
                            ) as rankrow
                        where rankrow.a <=5;""".format(gameDate, tuple(players)))
    # db.db.close()
    return pd.DataFrame(db.cursor.fetchall())


In [186]:
def get_keyplayer_game_record():
    db = DB.MYDB()
    df = pd.DataFrame()
    for season in season_keyplayer:
        for tcode in season_keyplayer[season]:
            pcode = season_keyplayer[season][tcode]
            if len(pcode) == 1:
                pcode.add('0')
            try:
            
                db.cursor.execute("""
                                SELECT * FROM hygp.player_record a join game_meta b on a.gmkey = b.gmkey
                                where (b.tcodeH = '{}' or b.tcodeA = '{}') and a.gmkey >= 'S17'
                                and a.pcode in {};
                                """.format(tcode, tcode, tuple(pcode)))
                data = pd.DataFrame(db.cursor.fetchall())
                df = pd.concat([df, data.groupby('gmkey').mean()], ignore_index=True)
            except Exception as e:
                print(season, tcode)
                print(pcode)
                print(data)
                input()
            # print(df)
        
    db.db.close()

    return df

In [290]:
db = DB.MYDB()
db.cursor.execute("SELECT gmkey, tcodeA, tcodeH, gameDate FROM hygp.game_meta where gmkey >= 'S17G01N1' order by gmkey;")
records = db.cursor.fetchall()
db.cursor.execute("SELECT gmkey, score, loss FROM hygp.team_record where home_away = 'H' and gmkey >= 'S17' order by gmkey;")
scores = db.cursor.fetchall()
db.cursor.execute("select tcode from team_code")
tcodes = db.cursor.fetchall()
tcodes = [tcode['tcode'] for tcode in tcodes]
# season_keyplayer = dict()
# for season in range(17, 41):
#     season_keyplayer[str(season)] = get_season_keyplayer(season)
db.db.close()
    

Connect!


In [188]:
def get_team_player(db):
    db.cursor.execute("select pcode, tcode, seasonCode from player;")
    total_players = pd.DataFrame(db.cursor.fetchall())
    players = dict()
    for season in range(17, 41):
        season_player = total_players[total_players['seasonCode'] == str(season)]
        players[str(season)] = dict()
        for tcode in tcodes: 
            players[str(season)][tcode] = tuple(season_player[season_player['tcode'] == tcode]['pcode'])
    return players


In [189]:
def get_recent_keyplayer(player, gameDate, db):
    db.cursor.execute("""
                    select *
                    from (
                        SELECT r.pcode, r.trueShooting, r.astRatio, r.rebRatio,r.home_away, m.gameDate, rank() over(partition by r.pcode order by gameDate desc) as a
                        FROM player_record r join game_meta m on r.gmkey = m.gmkey
                        WHERE m.gameDate < '{}'
                            and r.pcode in {}
                        ) as rankrow
                    where rankrow.a <=5;
                        """.format(gameDate, player))
    data = pd.DataFrame(db.cursor.fetchall()).groupby('pcode').mean()
    top_shoot = data.sort_values('trueShooting', ascending=False)[:1].index
    top_ast = data.sort_values('astRatio', ascending=False)[:1].index
    top_reb = data.sort_values('rebRatio', ascending=False)[:1].index

    top_player = set(top_shoot) | set(top_ast) | set(top_reb)

    return top_player


    

In [4]:
# def get_keyplayer_data_inmemory():
player_record = pd.read_csv('player_record.csv', delimiter=';')
db = DB.MYDB()
db.cursor.execute("select * from game_meta where gameDate >= '20101015'")
meta_data = db.cursor.fetchall()
for idx, meta in enumerate(meta_data):
    print(idx, meta)
    break
db.db.close()
# get_keyplayer_data_inmemory()

Connect!
0 [{'game_meta_idx': 6, 'gmkey': 'S17G01N1', 'gameCode': '01', 'gameDate': '20101015', 'tcodeA': '70', 'tcodeH': '10', 'weekDay': '금', 'gameStart': '1912', 'gameEnd': '2058', 'seasonCode': 17, 'seasonGrade': 1, 'seasonCategoryName': '정규시즌', 'seasonName': '2010-2011', 'stadiumname': '울산동천'}, {'game_meta_idx': 7, 'gmkey': 'S17G01N2', 'gameCode': '01', 'gameDate': '20101016', 'tcodeA': '65', 'tcodeH': '35', 'weekDay': '토', 'gameStart': '1501', 'gameEnd': '1706', 'seasonCode': 17, 'seasonGrade': 1, 'seasonCategoryName': '정규시즌', 'seasonName': '2010-2011', 'stadiumname': '잠실실내'}, {'game_meta_idx': 8, 'gmkey': 'S17G01N3', 'gameCode': '01', 'gameDate': '20101016', 'tcodeA': '60', 'tcodeH': '06', 'weekDay': '토', 'gameStart': '1500', 'gameEnd': '1659', 'seasonCode': 17, 'seasonGrade': 1, 'seasonCategoryName': '정규시즌', 'seasonName': '2010-2011', 'stadiumname': '부산사직'}, {'game_meta_idx': 9, 'gmkey': 'S17G01N4', 'gameCode': '01', 'gameDate': '20101016', 'tcodeA': '16', 'tcodeH': '30', 'week

In [335]:
def get_keyplayer_data(start=0, verborse=False):
    db = DB.MYDB()
    db.cursor.execute("select * from game_meta where gameDate >= '20101015'")
    meta_data = db.cursor.fetchall()

    season = 0

    df = pd.DataFrame()
    for idx, meta in enumerate(meta_data[start:]):
        print(idx, meta['gmkey'])
        gmkey = meta['gmkey']
        season_key = gmkey[1:3]
        if season != season_key:
            season = season_key
            db.cursor.execute("select pcode, home_away from player_record where gmkey like 'S{}%'".format(season))
            player = pd.DataFrame(db.cursor.fetchall())
            print(season_key)
        if verborse:
            print(player)
        h_player = tuple(player[player['home_away'] == '1']['pcode'])
        a_player = tuple(player[player['home_away'] == '2']['pcode'])
        gameDate = meta['gameDate']

        # key_player =  tuple(get_recent_keyplayer(player, gameDate, db))
        key_player =  tuple(get_recent_keyplayer(h_player, gameDate, db) | get_recent_keyplayer(a_player, gameDate, db))

        db.cursor.execute("""
                    select avg(ast) as ast, avg(bs) as bs, avg(defr) as defr, avg(dk) as dk, avg(dkA) as dkA, avg(ef) as ef, 
                            avg(fb) as fb, avg(fg) as fg, avg(fgA) as fgA, avg(fgtA) as fgtA, avg(fo) as fo, avg(foul) as foul, 
                            avg(ft) as ft, avg(ftA) as ftA, avg(gd) as gd, avg(offr) as offr, avg(pp) as pp, avg(ppA) as ppA, avg(pscore) as pscore, 
                            avg(stl) as stl, avg(tf) as tf, avg(threep) as threep, avg(threepA) as threepA, avg(r.to) as turonover, avg(wft) as wft, 
                            avg(woft) as woft, avg(rb) as rb, avg(score) as score, avg(trueShooting) as trueShooting, avg(astRatio) as astRatio, avg(rebRatio) as rebRatio,
                            home_away
                            from player_record r
                            where gmKey = '{}' and pcode in {}
                            group by home_away
        """.format(gmkey, key_player))
        data = db.cursor.fetchall()
        if verborse:
            print(data)
        for d in data:
            d['gmkey'] = gmkey
            if d['home_away'] == '1':
                d['tcode'] = meta['tcodeH']
            else:
                d['tcode'] = meta['tcodeA']

        data = pd.DataFrame(data)
        df = pd.concat([df, data], ignore_index=True)
    df.to_csv('key_player_recent5.csv')
    df.to_csv('test.csv')
    db.db.close()

# get_keyplayer_data(start=1265, verborse=True)
get_keyplayer_data()

Connect!
0 S17G01N1
17
1 S17G01N2
2 S17G01N3
3 S17G01N4
4 S17G01N5
5 S17G01N6
6 S17G01N7
7 S17G01N8
8 S17G01N9
9 S17G01N10
10 S17G01N11
11 S17G01N12
12 S17G01N13
13 S17G01N14
14 S17G01N15
15 S17G01N16
16 S17G01N17
17 S17G01N18
18 S17G01N19
19 S17G01N20
20 S17G01N21
21 S17G01N22
22 S17G01N23
23 S17G01N24
24 S17G01N25
25 S17G01N26
26 S17G01N27
27 S17G01N28
28 S17G01N29
29 S17G01N30
30 S17G01N31
31 S17G01N32
32 S17G01N33
33 S17G01N34
34 S17G01N35
35 S17G01N36
36 S17G01N37
37 S17G01N38
38 S17G01N39
39 S17G01N40
40 S17G01N41
41 S17G01N42
42 S17G01N43
43 S17G01N44
44 S17G01N45
45 S17G01N46
46 S17G01N47
47 S17G01N48
48 S17G01N49
49 S17G01N50
50 S17G01N51
51 S17G01N52
52 S17G01N53
53 S17G01N54
54 S17G01N55
55 S17G01N56
56 S17G01N57
57 S17G01N58
58 S17G01N59
59 S17G01N60
60 S17G01N61
61 S17G01N62
62 S17G01N63
63 S17G01N64
64 S17G01N65
65 S17G01N66
66 S17G01N67
67 S17G01N68
68 S17G01N69
69 S17G01N70
70 S17G01N71
71 S17G01N72
72 S17G01N73
73 S17G01N74
74 S17G01N75
75 S17G01N76
76 S17G01N77
77 S17

KeyboardInterrupt: 

In [195]:
def get_recent_team(team, gameDate, db):
    db.cursor.execute("""select b.*, a.gameDate 
                        from game_meta a, team_record b
                        where a.gmkey = b.gmkey and a.gameDate <'{}'
                            and b.tcode = '{}'
                        order by a.gameDate desc
                        limit 5""".format(gameDate, team))
    # db.db.close()
    return pd.DataFrame(db.cursor.fetchall())


In [196]:
def make_player_recent_data():
    h_data = []
    a_data = []
    db = DB.MYDB()
    for idx, (record, score) in enumerate(zip(records, scores)):
        gmkey = record['gmkey']
        gameDate = record['gameDate']
        season = gmkey[1:3]
        h_score = score['score']
        a_score = score['loss']
        tcodeH = record['tcodeH']
        tcodeA = record['tcodeA']
        if gmkey != score['gmkey']:
            print('index error', gmkey, score['gmkey'])
        if tcodeH == '75' or tcodeA == '75': #예외처리
            continue
        try:
            temp1 = dict(get_recent_players(season_keyplayer[season][tcodeH], gameDate, db).mean(numeric_only=True))
            temp1['score'] = h_score
            temp1['gmkey'] = gmkey
            temp2 = dict(get_recent_players(season_keyplayer[season][tcodeA], gameDate, db).mean(numeric_only=True))
            temp2['score'] = a_score
            temp2['gmkey'] = gmkey

            h_data.append(temp1)
            a_data.append(temp2)
        except Exception as e:
            print("error ", gmkey)
        if idx % 100 == 0:
            print(gmkey)
        

    h_df = pd.DataFrame(h_data).to_csv("h_data.csv")
    a_df = pd.DataFrame(h_data).to_csv("a_data.csv")
    # model = pd.concat([make_data(h_data, a_data), make_data(a_data, h_data)])
    # return model
    db.db.close()


In [197]:
def make_team_recent_data():
    h_data = []
    a_data = []
    db = DB.MYDB()
    for idx, (record, score) in enumerate(zip(records, scores)):
        gmkey = record['gmkey']
        gameDate = record['gameDate']
        h_score = score['score']
        a_score = score['loss']
        if gmkey != score['gmkey']:
            print('index error', gmkey, score['gmkey'])
        if record['tcodeH'] == '75' or record['tcodeA'] == '75': #예외처리
            continue
        try:
            temp1 = dict(get_recent_team(record['tcodeH'], gameDate, db).mean(numeric_only=True))
            temp1['score'] = h_score
            temp1['gmkey'] = gmkey
            temp2 = dict(get_recent_team(record['tcodeA'], gameDate, db).mean(numeric_only=True))
            temp2['score'] = a_score
            temp2['gmkey'] = gmkey

            h_data.append(temp1)
            a_data.append(temp2)
        except Exception as e:
            print("error ", gmkey)
            e.with_traceback()
        if idx % 100 == 0:
            print(gmkey)
        

    h_df = pd.DataFrame(h_data).to_csv("team_h_data_recent5.csv")
    a_df = pd.DataFrame(h_data).to_csv("team_a_data_recent5.csv")
    # model = pd.concat([make_data(h_data, a_data), make_data(a_data, h_data)])
    # return model
    db.db.close()

In [ ]:
# naive model
player_hdata = pd.read_csv('h_data.csv', index_col=0)
player_adata = pd.read_csv('a_data.csv', index_col=0)
# result = calculate_data(h_data,a_data)
# player_data = pd.concat([calculate_data(h_data,a_data), calculate_data(a_data,h_data)], ignore_index=True)

player_hdata.sort_values(by='gmkey', inplace=True)
player_adata.sort_values(by='gmkey', inplace=True)
team_hdata.sort_values(by='gmkey', inplace=True)
team_adata.sort_values(by='gmkey', inplace=True)

home_result = team_player_concat(calculate_data(team_hdata,team_adata), calculate_data(player_hdata, player_adata), 0)
away_result = team_player_concat(calculate_data(team_adata,team_hdata), calculate_data(player_adata, player_hdata), 0)
result = pd.concat([home_result, away_result], ignore_index=True)
# data1 = result.replace(0, 0.1)
data1 = result.fillna(0)
data1 = data1.drop(columns='gmkey')


In [319]:
#최근 5경기 기록으로 모델 생성
team_record = pd.read_csv('team_record_update_score.csv', index_col=0)
player_record = pd.read_csv('key_player_recent5_ver2.csv', index_col=0)

player_record = player_record.drop(index = [541, 3073, 3075, 3697, 3699, 6595, 7202, 7229])

team_record.sort_values('gmkey', inplace=True)
team_record.reset_index(inplace=True)
player_record.sort_values('gmkey', inplace=True)
player_record.reset_index(inplace=True)

h_team_data = team_record[team_record.index % 2 == 0].reset_index()
a_team_data = team_record[team_record.index % 2 != 0].reset_index()
h_player_data = team_record[team_record.index % 2 == 0].reset_index()
a_player_data = team_record[team_record.index % 2 != 0].reset_index()


In [303]:
scores

,gmkey,score,loss
0,S17G01N1,99,86
1,S17G01N10,90,78
2,S17G01N100,75,88
3,S17G01N101,92,90
4,S17G01N102,73,85
...,...,...,...
3647,S40G01N5,64,72
3648,S40G01N6,78,80
3649,S40G01N7,102,77
3650,S40G01N8,100,92


In [318]:
test = pd.read_csv("recent_avg_record_update3.csv", index_col=0)
# test 
h_test = test[test.index % 2 != 0].reset_index()
a_test = test[test.index % 2 == 0].reset_index()
scores = pd.DataFrame(scores)
h_test['score'] = scores['score']
h_test['loss'] = scores['loss']
a_test['score'] = scores['loss']
a_test['loss'] = scores['score']

d = pd.concat([h_test, a_test], ignore_index=True)
d.to_csv("team_record_update_score.csv")

,Unnamed: 0,gmkey,tcode,benchScoreCn,bf,bs,defr,dk,dkA,ef,...,tfb,threep,threepA,to,tto,turnoverScoreCn,wft,woft,stl,loss
2058,4369,S29G01N175,10,0.0,0.0,3.6,23.2,1.4,1.4,0.6,...,4.0,6.4,19.2,10.8,0.2,0.0,7.2,9.0,7.2,70
5706,4370,S29G01N175,55,0.0,0.0,2.8,26.2,0.6,0.6,0.6,...,5.0,6.8,21.4,11.2,0.6,0.0,6.6,9.8,4.8,71


In [322]:
team_data = pd.concat([calculate_data(h_team_data, a_team_data), calculate_data(a_team_data, h_team_data)], ignore_index=True)
player_data = pd.concat([calculate_data(h_player_data, a_player_data), calculate_data(a_player_data, h_player_data)], ignore_index=True)
team_data.to_csv('team_calculate_date.csv')
player_data.to_csv('player_calculate_date.csv')

data = team_player_concat(team_data,player_data, 0.5)
data.drop(columns='gmkey', inplace=True)
data.fillna(0, inplace=True)
data


,ast,twop,threep,freep,offr,ppp,dk,defr,stl,bs,foul,tf,gd,score
0,0.573333,0.589372,0.383562,0.745614,0.279661,0.645349,1.000000,0.783217,0.448276,0.076159,0.476440,0.000000,0.600000,99
1,0.568750,0.593607,0.428571,0.692982,0.240876,0.657754,1.000000,0.681529,0.500000,0.060317,0.458537,0.000000,0.454545,79
2,0.560811,0.597015,0.373333,0.695238,0.229508,0.674242,0.866667,0.600000,0.423077,0.057927,0.516432,0.857143,0.600000,79
3,0.464286,0.479167,0.396694,0.670732,0.281879,0.500000,0.777778,0.690323,0.544118,0.009677,0.540426,0.333333,0.333333,92
4,0.572464,0.502392,0.330000,0.737374,0.243056,0.569536,1.000000,0.694268,0.617647,0.058632,0.508264,0.400000,0.250000,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7291,0.589744,0.521505,0.343023,0.750000,0.272727,0.567376,0.000000,0.695364,0.481481,0.030137,0.567568,0.000000,0.636364,104
7292,0.524390,0.525000,0.380000,0.693333,0.151376,0.577320,0.500000,0.783951,0.605263,0.021875,0.503106,0.000000,0.230769,103
7293,0.583333,0.562232,0.360294,0.649351,0.349462,0.636872,1.000000,0.792746,0.413793,0.067989,0.490909,0.000000,0.500000,65
7294,0.634483,0.491803,0.312500,0.655172,0.316583,0.553191,0.000000,0.679245,0.602740,0.032448,0.510309,0.000000,0.538462,85


In [323]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
mean_squared_error(y_predict, y_test, squared = False)
model.score(X_test, y_test)



0.006739551445622882

In [332]:
features = X.columns.tolist()

tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)

# Extracting the importances by sklearn 
importances_sk = tree.feature_importances_

# Creating a dataframe with the feature importance by sklearn
feature_importance_sk = {}
for i, feature in enumerate(features):
    feature_importance_sk[feature] = round(importances_sk[i],5)
feature_importance_sk

{'ast': 0.07883,
 'twop': 0.08972,
 'threep': 0.07672,
 'freep': 0.0915,
 'offr': 0.08256,
 'ppp': 0.08034,
 'dk': 0.04158,
 'defr': 0.08288,
 'stl': 0.07612,
 'bs': 0.0902,
 'foul': 0.11199,
 'tf': 0.02462,
 'gd': 0.07294}

LinearRegression()

4.645847945014881

0.5510885556428042

('1234',)